# मेटा परिवारका मोडेलहरूसँग निर्माण गर्दै

## परिचय

यो पाठमा समेटिने विषयहरू:

- दुई मुख्य मेटा परिवारका मोडेलहरू - Llama 3.1 र Llama 3.2 को अन्वेषण
- प्रत्येक मोडेलका प्रयोगका क्षेत्र र परिदृश्य बुझ्ने
- प्रत्येक मोडेलका विशेषता देखाउने कोड उदाहरण

## मेटा परिवारका मोडेलहरू

यस पाठमा, हामी मेटा परिवार वा "Llama Herd" का २ मोडेलहरू - Llama 3.1 र Llama 3.2 को अन्वेषण गर्नेछौं।

यी मोडेलहरू विभिन्न भेरियन्टमा उपलब्ध छन् र Github Model marketplace मा पाइन्छन्। यहाँ Github Models प्रयोग गरेर [एआई मोडेलहरूसँग प्रोटोटाइप बनाउने](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) बारे थप जानकारी छ।

मोडेल भेरियन्टहरू:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*नोट: Llama 3 Github Models मा पनि उपलब्ध छ तर यो पाठमा समेटिने छैन*


## Llama 3.1

४०५ अर्ब प्यारामिटरहरूसँग, Llama 3.1 खुला स्रोत LLM श्रेणीमा पर्छ।

यो मोडेल पहिलेको Llama 3 को अपग्रेड हो, जसले निम्न सुधारहरू ल्याएको छ:

- ठूलो सन्दर्भ विन्डो - १२८k टोकन बनाम ८k टोकन
- बढी अधिकतम आउटपुट टोकन - ४०९६ बनाम २०४८
- अझ राम्रो बहुभाषिक समर्थन - तालिम टोकनको वृद्धि भएकोले

यी सुधारहरूले Llama 3.1 लाई GenAI एप्लिकेसनहरू बनाउँदा अझ जटिल प्रयोगका केसहरू सम्हाल्न सक्षम बनाउँछ, जस्तै:
- नेटिभ फंक्शन कलिंग - LLM वर्कफ्लो बाहिरका बाह्य टुल्स र फंक्शनहरू कल गर्न सक्ने क्षमता
- अझ राम्रो RAG प्रदर्शन - ठूलो सन्दर्भ विन्डोको कारण
- कृत्रिम डेटा उत्पादन - फाइन-ट्युनिङ जस्ता कार्यहरूको लागि प्रभावकारी डेटा सिर्जना गर्न सक्ने क्षमता


### नेटिभ फंक्शन कलिङ

Llama 3.1 लाई फंक्शन वा टुल कल गर्न अझ प्रभावकारी बनाउन फाइन-ट्युन गरिएको छ। यसमा दुईवटा बिल्ट-इन टुलहरू छन् जसलाई मोडेलले प्रयोगकर्ताको प्रॉम्प्ट अनुसार प्रयोग गर्न आवश्यक छ भनेर चिन्न सक्छ। यी टुलहरू हुन्:

- **Brave Search** - वेब सर्च गरेर मौसम जस्ता ताजा जानकारी प्राप्त गर्न प्रयोग गर्न सकिन्छ
- **Wolfram Alpha** - जटिल गणितीय गणनाका लागि प्रयोग गर्न सकिन्छ, जसले गर्दा आफ्नै फंक्शन लेख्न आवश्यक पर्दैन।

तपाईंले आफ्नै कस्टम टुलहरू पनि बनाउन सक्नुहुन्छ जसलाई LLM ले कल गर्न सक्छ।

तलको कोड उदाहरणमा:

- हामीले उपलब्ध टुलहरू (brave_search, wolfram_alpha) लाई सिस्टम प्रॉम्प्टमा परिभाषित गर्छौं।
- प्रयोगकर्ताको प्रॉम्प्ट पठाउँछौं जसले कुनै सहरको मौसमको बारेमा सोध्छ।
- LLM ले Brave Search टुलमा टुल कल गरेर प्रतिक्रिया दिनेछ, जुन यसरी देखिन्छ `<|python_tag|>brave_search.call(query="Stockholm weather")`

*नोट: यो उदाहरणले केवल टुल कल गर्छ, यदि तपाईंलाई नतिजा प्राप्त गर्न चाहिन्छ भने, तपाईंले Brave API पेजमा निःशुल्क खाता बनाउनुपर्नेछ र फंक्शन आफैं परिभाषित गर्नुपर्नेछ*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

एलएलएम भए तापनि, Llama 3.1 को एउटा सीमितता भनेको मल्टिमोडालिटी हो। अर्थात्, विभिन्न प्रकारका इनपुटहरू जस्तै तस्बिरहरूलाई प्रॉम्प्टको रूपमा प्रयोग गर्न र प्रतिक्रिया दिन सक्ने क्षमता। यो क्षमता Llama 3.2 का मुख्य विशेषताहरू मध्ये एक हो। यी विशेषताहरूमा समावेश छन्:

- मल्टिमोडालिटी - पाठ र तस्बिर दुवै प्रॉम्प्टको मूल्याङ्कन गर्न सक्ने क्षमता
- सानोदेखि मध्यम आकारका भेरिएसनहरू (11B र 90B) - यसले लचिलो डिप्लोयमेन्ट विकल्पहरू प्रदान गर्छ,
- केवल-पाठ भेरिएसनहरू (1B र 3B) - यसले मोडेललाई एज / मोबाइल डिभाइसहरूमा डिप्लोय गर्न र कम लेटेन्सी प्रदान गर्न सम्भव बनाउँछ

मल्टिमोडल समर्थनले खुला स्रोत मोडेलहरूको दुनियाँमा ठूलो फड्को प्रस्तुत गर्छ। तलको कोड उदाहरणले तस्बिर र पाठ दुवै प्रॉम्प्ट लिएर Llama 3.2 90B बाट तस्बिरको विश्लेषण प्राप्त गर्छ।

### Llama 3.2 सँग मल्टिमोडल समर्थन


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## सिकाइ यहाँ रोकिँदैन, यात्रा जारी राख्नुहोस्

यो पाठ समाप्त गरेपछि, हाम्रो [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) हेर्नुहोस् र आफ्नो Generative AI ज्ञान अझ बढाउनुहोस्!



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटि वा अशुद्धि हुन सक्छ। मूल भाषा भएको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
